In [ ]:
# PyTorch 神经网络


In [3]:
# 定义网络
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1=nn.Conv2d(1,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        
        # an affine operation:y=Wx+b
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    
    def forward(self,x):
        # 2X2 max pooling
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # 如果是方阵,则可以只使用一个数字进行定义
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size=x.size()[1:]# 除去批处理维度的其他所有维度
        num_features=1
        for s in size:
            num_features *=s
        return num_features

In [16]:
net =Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
params=list(net.parameters())
print(len(params))
print(params[0].size()) #conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [19]:
# 尝试一个随机的 32x32 的输入。注意:这个网络 (LeNet）的期待输入是 32x32 的张量。如果使用 MNIST 数据集来训练这个网络，要把图片大小重新调整到 32x32
input =torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[-0.0857,  0.1251, -0.0558,  0.0736,  0.0611, -0.0085,  0.1574,  0.0733,
          0.1089,  0.1139]], grad_fn=<AddmmBackward>)


In [20]:
# 清零所有参数的梯度缓存，然后进行随机梯度的反向传播
net.zero_grad()
out.backward(torch.randn(1,10))

In [23]:
# 一个损失函数接受一对(output, target)作为输入，计算一个值来估计网络的输出和目标值相差多少。

# nn包中有很多不同的损失函数。nn.MSELoss是比较简单的一种，它计算输出和目标的均方误差(mean-squared error）。
output=net(input)
target=torch.randn(10) # 本例子中使用模拟数据
print("原值")
print(target)
target=target.view(1,-1) #使目标值与数据值尺寸一致
print("修正后值")
print(target)
criterion=nn.MSELoss() #是比较简单的一种，它计算输出和目标的均方误差

loss = criterion(output,target)
print(loss)

原值
tensor([ 0.1739,  0.2270, -0.6012,  0.1491, -0.3252, -0.4563,  0.4872, -0.1345,
        -1.0956, -0.6316])
修正后值
tensor([[ 0.1739,  0.2270, -0.6012,  0.1491, -0.3252, -0.4563,  0.4872, -0.1345,
         -1.0956, -0.6316]])
tensor(0.2889, grad_fn=<MseLossBackward>)


In [24]:
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
    #   -> view -> linear -> relu -> linear -> relu -> linear
    #   -> MSELoss
    #   -> loss
print(loss.grad_fn) #MSELoss

In [25]:
print(loss.grad_fn.next_functions[0][0]) #Linear

In [26]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [27]:
# 反向传播
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0076,  0.0091,  0.0014,  0.0014,  0.0007, -0.0045])


In [29]:
# 更新权重
# 最简单的更新规则是随机梯度下降法 (SGD）:

# weight = weight - learning_rate * gradient
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [36]:
import torch.optim as optim

# 创建优化器(optimizer）
optimizer = optim.SGD(net.parameters(),lr=0.01)

#在训练的迭代中：
optimizer.zero_grad() #清零梯度缓存
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step() #更新参数